In [1]:
import pandas as pd
import quandl

In [8]:
#get Quandl private key
quandl_private_key = open('private/quandlKey.txt', 'r').read()

In [11]:
#start and end date of the analysis
start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2018-12-22')

In [12]:
portfolio_list = ['SPTL', 'THOMASCOOK', 'TATAMTRDVR', 'ASHIANA']

In [24]:
portfolio_close_price = pd.DataFrame()

for stock in portfolio_list:
    portfolio_close_price[stock] = quandl.get("NSE/{}.5".format(stock), authtoken=quandl_private_key, start_date=start_date, end_date=end_date)['Close']

In [25]:
portfolio_close_price.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 340 entries, 2017-08-08 to 2018-12-21
Data columns (total 4 columns):
SPTL          340 non-null float64
THOMASCOOK    340 non-null float64
TATAMTRDVR    340 non-null float64
ASHIANA       340 non-null float64
dtypes: float64(4)
memory usage: 13.3 KB


In [26]:
portfolio_close_price.describe()

,SPTL,THOMASCOOK,TATAMTRDVR,ASHIANA
count,340.000000,340.000000,340.000000,340.000000
mean,60.089118,248.061765,182.644706,155.623382
std,26.042453,21.683914,50.753493,21.115043
min,21.500000,194.950000,85.900000,116.100000
25%,37.750000,233.887500,142.312500,135.500000
50%,58.575000,244.725000,190.225000,160.375000
75%,80.712500,269.300000,228.375000,172.650000
max,123.500000,292.150000,259.650000,199.500000


In [27]:
portfolio_close_price.head()

,SPTL,THOMASCOOK,TATAMTRDVR,ASHIANA
Date,,,,
2017-08-08,123.50,223.90,248.45,187.55
2017-08-09,117.35,223.50,242.25,185.25
2017-08-10,111.50,221.15,221.45,179.10
2017-08-11,105.95,219.75,223.20,172.15
2017-08-14,107.60,218.60,226.45,171.75
